In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten, BatchNormalization, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

In [ ]:
df_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
df_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.label.unique()

In [ ]:
from matplotlib.pyplot import imshow

width=5
height=5
rows = 3
cols = 3
axes=[]

fig=plt.figure()
fig.set_size_inches(6,8)

for i in range(rows*cols):
    sample=np.reshape(df_train[df_train.columns[1:]].iloc[i].values/255,(28,28))
    axes.append(fig.add_subplot(rows,cols,i+1))
    plt.title("Labeled class : {}".format(df_train["label"].iloc[i]))
    plt.imshow(sample, 'gray')
fig.tight_layout()
plt.show()

In [ ]:
#Explanatory Data Analysis

plt.figure(figsize=(8,6))
ax = sns.countplot(x='label',data=df_train)

plt.title("Label Distribution")
total= len(df_train.label)
for p in ax.patches:
    percentage = f'{100 * p.get_height() / total:.1f}%\n'
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ax.annotate(percentage, (x, y), ha='center', va='center')

In [ ]:
# separate target values from df_train
targets = df_train.label
features = df_train.drop("label",axis=1)

**PreProcessing**

In [ ]:
features = features/255
df_test = df_test/255

In [ ]:
X_train , X_val , Y_train , Y_val = train_test_split(features , targets , test_size=0.2 ,random_state = 1)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(Y_train,10)
y_val =  to_categorical(Y_val,10)

print(y_train[2])
print(len(y_train))

In [ ]:
X_train = X_train.values.reshape(len(X_train),28,28,1)
X_val = X_val.values.reshape(len(X_val),28,28,1)

df_test = df_test.values.reshape(len(df_test),28,28,1)

input_shape = (28,28,1)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential()

#Conv1
model.add(tf.keras.layers.Conv2D(32 , 3 , input_shape=input_shape , activation = 'relu' , padding = 'same') )
model.add(tf.keras.layers.MaxPool2D(2))

#Conv2
model.add(tf.keras.layers.Conv2D(64 , 3  , activation = 'relu' , padding = 'same') )
model.add(tf.keras.layers.MaxPool2D(2))

#Conv3
model.add(tf.keras.layers.Conv2D(128 , 3  , activation = 'relu' , padding = 'same') )
model.add(tf.keras.layers.MaxPool2D(2))
model.add(tf.keras.layers.Dropout(0.3))

#Conv4
model.add(tf.keras.layers.Conv2D(128 , 3  , activation = 'relu' , padding = 'same') )
model.add(tf.keras.layers.MaxPool2D(2))
model.add(tf.keras.layers.Dropout(0.25))


model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256 , activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(10 , activation = 'softmax'))


model.summary()

In [ ]:
#Take a look at any layer in the model by changing layer_name below.

layer_name = 'conv2d_2'
layer_dict = {layer.name : layer for layer in model.layers}
modelslice = tf.keras.Model(model.inputs, layer_dict[layer_name].output)
image = np.reshape(df_train[df_train.columns[1:]].iloc[2].values/255,(28,28))
image = np.expand_dims(image, axis=0)
feature_maps = modelslice.predict(image)
plt.figure(figsize=(20, 10))

for i in range(16): # This is itterating through the 16 filter of the convolution!
    plt.subplot(4,8,i+1)
    plt.axis('off')
    plt.imshow(feature_maps[0, :, :, i-1], cmap='inferno')
    plt.tight_layout(pad=0.0)

In [ ]:
# COMPILING
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adamax' , metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train , y_train , batch_size = 42 , epochs = 30 , validation_data = (X_val,y_val))

In [ ]:
fig = plt.figure(figsize=(20,12))

plt.subplot(211)
plt.style.use('dark_background')
plt.plot(history.history['accuracy'], color='c', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r', label="Validation accuracy")

plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(loc='best')

plt.subplot(212)
plt.style.use('dark_background')
plt.plot(history.history['loss'], color='m', label="Training loss")
plt.plot(history.history['val_loss'], color='r', label="Validation loss")
plt.ylabel('Loss')
plt.legend(loc='best')

In [ ]:
predictions = model.predict(df_test)
predictions = [np.argmax(a) for a in predictions]

In [ ]:
output = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
output['Label'] = predictions
output.to_csv('submission.csv',index=False)